# Technisches Assessment | 05.07.2024 | Sebastian Archila

## Libraries

In [1]:

# Standard library imports

# Local imports
import pandas as pd
import re

## Inhaltsverzeichnis

<p><a href="#1-extract">1. Extract</a></p>
<p style="margin-left: 20px;"><a href="#11-data-check">1.1. Data check</a></p>
<p style="margin-left: 20px;"><a href="#12-general-analysis">1.2. General analysis</a></p>
<p style="margin-left: 20px;"><a href="#13-data-cleaning">1.3. Data cleaning</a></p>

## 1. Extract

### 1.1. Data check

In [2]:
# Load and create df
data_path = "data/zugfahrten.csv"
try:
    zugfahrten_df = pd.read_csv(data_path)
    print("Data loaded successfully.")
except FileNotFoundError:
    print("File not found. Please provide a valid path.")
except Exception as e:
    print("An error occurred while loading the data:", e)

Data loaded successfully.


### 1.2. General analysis

In [3]:
# Explore data
print("Data shape:", zugfahrten_df.shape)
print("Column names:", zugfahrten_df.columns)

# Check the first few rows
zugfahrten_df.head()

Data shape: (26250, 12)
Column names: Index(['Fahrtnummer', 'Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum',
       'Messung', 'Zug id', 'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km',
       'Passagieranzahl', 'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,3.094127e+09,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
1,3.094127e+09,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
2,3.094127e+09,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
3,3.094127e+09,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
4,3.094127e+09,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam


In [4]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26250 entries, 0 to 26249
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fahrtnummer      26212 non-null  float64
 1   Fahrzeugnummer   26206 non-null  float64
 2   Buchungskreis    26209 non-null  object 
 3   Abfahrtsdatum    26216 non-null  object 
 4   Messung          26208 non-null  float64
 5   Zug id           26203 non-null  object 
 6   Abfahrtszeit     26198 non-null  object 
 7   Ankunftszeit     26218 non-null  object 
 8   Entfernung km    26201 non-null  object 
 9   Passagieranzahl  26212 non-null  object 
 10  Abfahrtsbahnhof  26215 non-null  object 
 11  Ankunftsbahnhof  26205 non-null  object 
dtypes: float64(3), object(9)
memory usage: 2.4+ MB


In [5]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrtnummer', 'Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum',
       'Messung', 'Zug id', 'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km',
       'Passagieranzahl', 'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer        38
Fahrzeugnummer     44
Buchungskreis      41
Abfahrtsdatum      34
Messung            42
Zug id             47
Abfahrtszeit       52
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


### 1.3. Data cleaning

#### Zug id

In [6]:
# List of unique values in Zug id
print("\nUnique values in Zug id:")
print(zugfahrten_df['Zug id'].unique())


Unique values in Zug id:
['3094127253' '8322069260' '9523145550' ... 'BJOUTUFYHN' 'JAUBUQTFAN'
 'OANNRSMDFS']


In [7]:
# Find rows where Zug id is missing (NaN or empty string after conversion)
missing_zug_id_rows = zugfahrten_df[zugfahrten_df['Zug id'].isnull() | (zugfahrten_df['Zug id'] == '')]

# Display these rows
print("Rows where Zug id is missing:")
display(missing_zug_id_rows.head())

Rows where Zug id is missing:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
1144,4.361218e+09,9.0,1004,2023-02-13,983.70,NaN,2024-06-08 13:34:40,2024-06-08 18:19:40,836,164,Jeremyberg,East Nancychester
2977,2.035955e+08,9.0,XXXX,2020-05-14,3582.34,NaN,2024-06-05 04:26:18,2024-06-05 07:08:18,688,230,South Jessicaville,New Gabriellaside
3461,2.914621e+09,1.0,1002,03/07/1998,2379.73,NaN,2024-05-27 07:04:47,2024-05-27 10:24:47,777,202,Lake Paulview,Joanchester
3556,6.373607e+09,5.0,XXXX,2021-12-11,3109.16,NaN,2024-06-08 08:07:57,2024-06-08 12:34:57,901,492,Thomaston,Martinezfurt
5039,7.338116e+09,9.0,1008,2023-05-02,1774.81,NaN,2024-06-22 23:01:58,2024-06-23 02:46:58,213,147,West Cathy,Whitestad


In [8]:
# Impute missing values in Zug id - loop 1
def impute_zug_id(df):
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'Zug id']) or df.loc[i, 'Zug id'] == 'nan':  # Checking for 'nan' as it's the string representation
            # Check the Zug id value one row before
            if i >= 1 and pd.notna(df.loc[i-1, 'Zug id']) and df.loc[i-1, 'Zug id'] != 'nan':
                if (df.loc[i-1, 'Passagieranzahl'] == df.loc[i, 'Passagieranzahl'] and
                    df.loc[i-1, 'Abfahrtsbahnhof'] == df.loc[i, 'Abfahrtsbahnhof'] and
                    df.loc[i-1, 'Ankunftsbahnhof'] == df.loc[i, 'Ankunftsbahnhof']):
                    df.loc[i, 'Zug id'] = df.loc[i-1, 'Zug id']
            # If the row before is not valid, check the row after
            elif i < len(df) - 1 and pd.notna(df.loc[i+1, 'Zug id']) and df.loc[i+1, 'Zug id'] != 'nan':
                if (df.loc[i+1, 'Passagieranzahl'] == df.loc[i, 'Passagieranzahl'] and
                    df.loc[i+1, 'Abfahrtsbahnhof'] == df.loc[i, 'Abfahrtsbahnhof'] and
                    df.loc[i+1, 'Ankunftsbahnhof'] == df.loc[i, 'Ankunftsbahnhof']):
                    df.loc[i, 'Zug id'] = df.loc[i+1, 'Zug id']
    return df

# Apply the imputation function
zugfahrten_df = impute_zug_id(zugfahrten_df)

In [9]:
# Impute missing values in Zug id - loop 2
def impute_zug_id(df):
    for i in range(len(df)):
        if pd.isna(df.loc[i, 'Zug id']) or df.loc[i, 'Zug id'] == 'nan':  # Checking for NaN or 'nan'
            # Check the Zug id value one row before
            if i >= 1 and pd.notna(df.loc[i-1, 'Zug id']):
                if (df.loc[i-1, 'Passagieranzahl'] == df.loc[i, 'Passagieranzahl'] and
                    df.loc[i-1, 'Abfahrtsbahnhof'] == df.loc[i, 'Abfahrtsbahnhof'] and
                    df.loc[i-1, 'Abfahrtszeit'] == df.loc[i, 'Abfahrtszeit'] and
                    df.loc[i-1, 'Ankunftsbahnhof'] == df.loc[i, 'Ankunftsbahnhof']):
                    df.loc[i, 'Zug id'] = df.loc[i-1, 'Zug id']
            # Check the Zug id value one row after if still missing
            if pd.isna(df.loc[i, 'Zug id']):
                if i < len(df) - 1 and pd.notna(df.loc[i+1, 'Zug id']):
                    if (df.loc[i+1, 'Passagieranzahl'] == df.loc[i, 'Passagieranzahl'] and
                        df.loc[i+1, 'Abfahrtsbahnhof'] == df.loc[i, 'Abfahrtsbahnhof'] and
                        df.loc[i+1, 'Abfahrtszeit'] == df.loc[i, 'Abfahrtszeit'] and
                        df.loc[i+1, 'Ankunftsbahnhof'] == df.loc[i, 'Ankunftsbahnhof']):
                        df.loc[i, 'Zug id'] = df.loc[i+1, 'Zug id']
    return df

# Apply the imputation function
zugfahrten_df = impute_zug_id(zugfahrten_df)

In [10]:
# Find rows where Zug id is missing (NaN or empty string after conversion)
missing_zug_id_rows = zugfahrten_df[zugfahrten_df['Zug id'].isnull() | (zugfahrten_df['Zug id'] == '')]

# Display these rows
print("Rows where Zug id is missing:")
display(missing_zug_id_rows)

Rows where Zug id is missing:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25020,8.667541e+09,2.0,1007,24/12/2019,3014.11,NaN,2024-06-12 22:52:13,2024-06-13 02:25:13,882,268,West Jasonfort,Katherinetown
25884,7.699680e+09,5.0,1003,2023-03-25,3790.86,NaN,2024-05-28 07:15:17,2024-05-28 12:10:17,270,279,Duranfort,Hernandezshire
26229,8.480904e+09,11.0,1000,2021-02-24,4603.09,NaN,2024-05-29 11:11:48,2024-05-29 11:45:48,319,359,Amandaland,Lake Wendy


In [11]:
# Count the number of rows to be deleted
num_rows_deleted = missing_zug_id_rows.shape[0]

# Drop the rows with missing Zug id values
zugfahrten_df = zugfahrten_df.dropna(subset=['Zug id'])
zugfahrten_df = zugfahrten_df[zugfahrten_df['Zug id'] != '']

# Display the number of rows deleted
print(f"Number of rows deleted: {num_rows_deleted}")

Number of rows deleted: 3


<style>
    .conclusion-header {
        font-weight: bold;
        color: #BB0056;
        font-size: 1.5em;
        margin-bottom: 0.5em;
    }
    .conclusion-body {
        font-size: 1em;
        line-height: 1.5em;
        color: white;
    }
</style>

<div class="conclusion-header">Conclusion</div>
<div class="conclusion-body">
    Instead of deleting rows with missing values in <b>Zug id</b>, values were imputed. 
    Three rows still have missing values; however, those are repeated rows. 
    Therefore, just these three rows have been deleted.
</div>

In [12]:
# List of unique values in Zug id
print("\nUnique values in Zug id:")
print(zugfahrten_df['Zug id'].unique())


Unique values in Zug id:
['3094127253' '8322069260' '9523145550' ... 'BJOUTUFYHN' 'JAUBUQTFAN'
 'OANNRSMDFS']


In [13]:
# Define a regular expression pattern to match any string with a letter
pattern = re.compile(r'[a-zA-Z]')

# Find rows where Zug id contains letters
rows_with_letters = zugfahrten_df[zugfahrten_df['Zug id'].apply(lambda x: bool(pattern.search(x)))]

# Display these rows
print("Rows where Zug id contains letters:")
display(rows_with_letters)

Rows where Zug id contains letters:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
116,4.892646e+09,4.0,1003,2024-04-18,2088.06,UYJGLIEQFP,2024-06-11 00:06:46,2024-06-11 01:14:46,637,89,New Christopherview,North Crystalmouth
481,7.811937e+09,1.0,1005,2023-09-29,4985.06,GPMEHEITKD,2024-06-24 13:55:29,2024-06-24 15:24:29,880,342,Jamesstad,Meyerborough
482,7.811937e+09,2.0,1005,2023-09-29,761.21,TTMKEMIVRH,2024-06-24 13:55:29,2024-06-24 15:24:29,880,342,Jamesstad,Meyerborough
948,6.238971e+09,4.0,1000,2022-11-05,3617.33,OUCPNBQKFY,2024-05-31 22:57:29,2024-06-01 01:08:29,256,123,Lake Dan,Richardborough
961,7.808581e+09,12.0,XXXX,2020-01-10,3116.10,UKCLIBYGSZ,2024-06-21 04:35:34,2024-06-21 05:28:34,459,364,Jacquelineland,South Luis
...,...,...,...,...,...,...,...,...,...,...,...,...
24692,1.097616e+09,2.0,1005,2024-04-27,1390.32,VYKANFWFSJ,2024-06-24 22:19:04,2024-06-25 00:09:04,854,278,North Amandaland,Catherinetown
24777,6.176395e+09,2.0,1006,2020-04-14,2881.34,WBPGKZZDFC,2024-05-27 07:59:38,2024-05-27 09:59:38,150,354,Millerton,Port Nicholeside
25409,4.190075e+09,4.0,1007,2021-08-27,4076.53,BJOUTUFYHN,2024-06-11 13:55:33,2024-06-11 17:54:33,824,489,South Karabury,Michaelhaven
25586,2.497838e+09,4.0,1000,2021-11-16,506.18,JAUBUQTFAN,2024-05-27 18:23:51,2024-05-27 19:59:51,76,293,Brandonland,North Lisa


In [14]:
# Define a regular expression pattern to check for any letter in the 'Zug id'
pattern = re.compile(r'[a-zA-Z]')
original_zugfahrten_df = zugfahrten_df.copy()

# Function to check and update 'Zug id'
def update_zug_id(index, direction):
    if 0 <= index + direction < len(zugfahrten_df):
        current_row = zugfahrten_df.iloc[index]
        adjacent_row = zugfahrten_df.iloc[index + direction]
        if (current_row['Buchungskreis'] == adjacent_row['Buchungskreis'] and
            current_row['Entfernung km'] == adjacent_row['Entfernung km'] and
            current_row['Abfahrtsbahnhof'] == adjacent_row['Abfahrtsbahnhof']):
            zugfahrten_df.at[index, 'Zug id'] = adjacent_row['Zug id']

# Function to perform a pass over the DataFrame
def perform_pass():
    for i, row in zugfahrten_df.iterrows():
        # Check if the current row's 'Zug id' contains letters
        if pattern.search(row['Zug id']):
            # Check and update from the previous row
            update_zug_id(i, -1)
            # Check and update from the next row
            update_zug_id(i, 1)

# First pass
perform_pass()

# Find rows where 'Zug id' was modified in the first pass
modified_rows_first_pass = zugfahrten_df[zugfahrten_df['Zug id'] != original_zugfahrten_df['Zug id']]

# Display the modified rows from the first pass
print("Rows that were modified in the first pass:")
print(modified_rows_first_pass)

# Second pass to recheck and re-update 'Zug id' based on the modified DataFrame
perform_pass()

# Find rows where 'Zug id' was modified in the second pass
modified_rows_second_pass = zugfahrten_df[zugfahrten_df['Zug id'] != original_zugfahrten_df['Zug id']]

# Display the modified rows from the second pass
print("Rows that were modified in the second pass:")
print(modified_rows_second_pass)

Rows that were modified in the first pass:
        Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
116    4.892646e+09             4.0          1003    2024-04-18  2088.06   
481    7.811937e+09             1.0          1005    2023-09-29  4985.06   
482    7.811937e+09             2.0          1005    2023-09-29   761.21   
948    6.238971e+09             4.0          1000    2022-11-05  3617.33   
961    7.808581e+09            12.0          XXXX    2020-01-10  3116.10   
...             ...             ...           ...           ...      ...   
24506  6.128839e+09            15.0          1007    2024-06-12  4033.48   
24619  7.935409e+09             6.0          XXXX    2021-01-23  4364.79   
24663  6.951694e+09             1.0          1001    2023-09-23  1232.80   
24692  1.097616e+09             2.0          1005    2024-04-27  1390.32   
24777  6.176395e+09             2.0          1006    2020-04-14  2881.34   

           Zug id         Abfahrtszeit      

In [15]:
# Save imputed_df to a CSV file
zugfahrten_df.to_csv('zugfahrten_df.csv', index=False)

In [16]:
# Define a regular expression pattern to match any string with a letter
pattern = re.compile(r'[a-zA-Z]')

# Find rows where Zug id contains letters
rows_with_letters = zugfahrten_df[zugfahrten_df['Zug id'].apply(lambda x: bool(pattern.search(x)))]
zugfahrten_df = zugfahrten_df[~zugfahrten_df.index.isin(rows_with_letters.index)]

# Display these rows
print("Rows where Zug id contains letters:")
display(rows_with_letters)
print("-------------------")
print("New df:")
display(zugfahrten_df)

Rows where Zug id contains letters:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25409,4.190075e+09,4.0,1007,2021-08-27,4076.53,BJOUTUFYHN,2024-06-11 13:55:33,2024-06-11 17:54:33,824,489,South Karabury,Michaelhaven
25586,2.497838e+09,4.0,1000,2021-11-16,506.18,JAUBUQTFAN,2024-05-27 18:23:51,2024-05-27 19:59:51,76,293,Brandonland,North Lisa
25815,3.808503e+09,7.0,1002,2023-07-28,4260.37,OANNRSMDFS,2024-05-29 02:35:26,2024-05-29 07:25:26,937,247,East Jason,Port Evanview


-------------------
New df:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,3.094127e+09,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
1,3.094127e+09,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
2,3.094127e+09,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
3,3.094127e+09,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
4,3.094127e+09,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
...,...,...,...,...,...,...,...,...,...,...,...,...
26245,1.551875e+09,15.0,1004,2024-04-30,3778.08,1551875283,2024-06-22 09:35:21,2024-06-22 10:46:21,560,398,Lake Adam,Johnsonbury
26246,2.685583e+09,5.0,1009,2021-08-20,4410.33,2685582618,2024-05-27 01:52:13,2024-05-27 06:15:13,640,478,North Cassandra,West Nicole
26247,7.811937e+09,2.0,1005,2023-09-29,761.21,7811937135,2024-06-24 13:55:29,2024-06-24 15:24:29,880,342,Jamesstad,Meyerborough
26248,4.366267e+09,15.0,1000,2020-01-28,3140.46,4366266646,2024-06-15 11:53:47,2024-06-15 13:39:47,576,149,South Michael,Hayesfort


<style>
    .conclusion-header {
        font-weight: bold;
        color: #BB0056;
        font-size: 1.5em;
        margin-bottom: 0.5em;
    }
    .conclusion-body {
        font-size: 1em;
        line-height: 1.5em;
        color: white;
    }
</style>

<div class="conclusion-header">Conclusion</div>
<div class="conclusion-body">
    Instead of deleting rows with alphabetic characters in the Zug id, the values were corrected. 
    However, three rows still contained alphabetic characters due to being duplicate entries. 
    Consequently, only these three duplicate rows were deleted.
</div>

#### Fahrtnummer

In [17]:
zugfahrten_df = zugfahrten_df.drop(columns=['Fahrtnummer'])

# Display the DataFrame after deleting the column
print("DataFrame after deleting 'Fahrtnummer' column:")
display(zugfahrten_df.head())

DataFrame after deleting 'Fahrtnummer' column:


,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
1,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
2,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
3,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
4,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam


In [18]:
# Copy the 'Zug id' column and name it 'Fahrtnummer'
zugfahrten_df.insert(0, 'Fahrtnummer', zugfahrten_df['Zug id'])

# Display the DataFrame after copying the column
print("DataFrame after copying 'Zug id' column to 'Fahrtnummer':")
display(zugfahrten_df.head())

DataFrame after copying 'Zug id' column to 'Fahrtnummer':


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,3094127253,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
1,3094127253,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
2,3094127253,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
3,3094127253,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
4,3094127253,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam


<style>
    .conclusion-header {
        font-weight: bold;
        color: #BB0056;
        font-size: 1.5em;
        margin-bottom: 0.5em;
    }
    .conclusion-body {
        font-size: 1em;
        line-height: 1.5em;
        color: white;
    }
</style>

<div class="conclusion-header">Conclusion</div>
<div class="conclusion-body">
    This was done to maintain the correct format and to preserve the leading "0" in the IDs where required.
</div>

In [19]:
line_88 = zugfahrten_df.iloc[87]  # iloc is zero-indexed, so line 88 is at position 87
print("Row at line 88:")
print(line_88)

Row at line 88:
Fahrtnummer                 0334669292
Fahrzeugnummer                    14.0
Buchungskreis                     1009
Abfahrtsdatum               2023-05-21
Messung                        4037.86
Zug id                      0334669292
Abfahrtszeit       2024-06-22 08:50:13
Ankunftszeit       2024-06-22 12:19:13
Entfernung km                      459
Passagieranzahl                    442
Abfahrtsbahnhof       Port Roberthaven
Ankunftsbahnhof              New Paula
Name: 87, dtype: object


In [20]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26244 entries, 0 to 26249
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fahrtnummer      26244 non-null  object 
 1   Fahrzeugnummer   26200 non-null  float64
 2   Buchungskreis    26203 non-null  object 
 3   Abfahrtsdatum    26210 non-null  object 
 4   Messung          26202 non-null  float64
 5   Zug id           26244 non-null  object 
 6   Abfahrtszeit     26192 non-null  object 
 7   Ankunftszeit     26212 non-null  object 
 8   Entfernung km    26195 non-null  object 
 9   Passagieranzahl  26206 non-null  object 
 10  Abfahrtsbahnhof  26209 non-null  object 
 11  Ankunftsbahnhof  26199 non-null  object 
dtypes: float64(2), object(10)
memory usage: 2.6+ MB


In [21]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum', 'Messung',
       'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer     44
Buchungskreis      41
Abfahrtsdatum      34
Messung            42
Zug id              0
Abfahrtszeit       52
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


In [22]:
zugfahrten_df

,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,3094127253,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
1,3094127253,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
2,3094127253,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
3,3094127253,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
4,3094127253,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688,470,Millerside,Lake Adam
...,...,...,...,...,...,...,...,...,...,...,...,...
26245,1551875283,15.0,1004,2024-04-30,3778.08,1551875283,2024-06-22 09:35:21,2024-06-22 10:46:21,560,398,Lake Adam,Johnsonbury
26246,2685582618,5.0,1009,2021-08-20,4410.33,2685582618,2024-05-27 01:52:13,2024-05-27 06:15:13,640,478,North Cassandra,West Nicole
26247,7811937135,2.0,1005,2023-09-29,761.21,7811937135,2024-06-24 13:55:29,2024-06-24 15:24:29,880,342,Jamesstad,Meyerborough
26248,4366266646,15.0,1000,2020-01-28,3140.46,4366266646,2024-06-15 11:53:47,2024-06-15 13:39:47,576,149,South Michael,Hayesfort


### Fahrzeugnummer

In [23]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum', 'Messung',
       'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer     44
Buchungskreis      41
Abfahrtsdatum      34
Messung            42
Zug id              0
Abfahrtszeit       52
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


In [24]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 26244 entries, 0 to 26249
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fahrtnummer      26244 non-null  object 
 1   Fahrzeugnummer   26200 non-null  float64
 2   Buchungskreis    26203 non-null  object 
 3   Abfahrtsdatum    26210 non-null  object 
 4   Messung          26202 non-null  float64
 5   Zug id           26244 non-null  object 
 6   Abfahrtszeit     26192 non-null  object 
 7   Ankunftszeit     26212 non-null  object 
 8   Entfernung km    26195 non-null  object 
 9   Passagieranzahl  26206 non-null  object 
 10  Abfahrtsbahnhof  26209 non-null  object 
 11  Ankunftsbahnhof  26199 non-null  object 
dtypes: float64(2), object(10)
memory usage: 2.6+ MB


In [25]:
# Check for duplicate rows based on all columns
duplicate_rows = zugfahrten_df[zugfahrten_df.duplicated()]

# Display the duplicate rows
print("Duplicate rows based on all columns:")
display(duplicate_rows)

Duplicate rows based on all columns:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25000,6703658983,10.0,XXXX,2024-06-10,2333.12,6703658983,2024-06-09 21:40:44,2024-06-10 01:08:44,192,350,New Alexachester,East Annette
25001,3855774328,12.0,1003,2023-12-08,2914.65,3855774328,2024-06-19 15:47:24,2024-06-19 19:09:24,83,244,East Susanbury,Meyerstad
25002,4966654600,9.0,1001,2024-03-02,884.01,4966654600,2024-06-21 03:51:52,2024-06-21 05:54:52,505,394,Port Brittneytown,Jenniferbury
25003,2924503871,5.0,1003,2020-06-11,3465.87,2924503871,2024-06-08 04:28:15,2024-06-08 09:00:15,219,310,Williamtown,Tanyafurt
25004,7146049531,3.0,XXXX,2022-10-22,4469.59,7146049531,2024-05-31 02:52:58,2024-05-31 04:47:58,812,203,New Jillhaven,Lake Thomasview
...,...,...,...,...,...,...,...,...,...,...,...,...
26245,1551875283,15.0,1004,2024-04-30,3778.08,1551875283,2024-06-22 09:35:21,2024-06-22 10:46:21,560,398,Lake Adam,Johnsonbury
26246,2685582618,5.0,1009,2021-08-20,4410.33,2685582618,2024-05-27 01:52:13,2024-05-27 06:15:13,640,478,North Cassandra,West Nicole
26247,7811937135,2.0,1005,2023-09-29,761.21,7811937135,2024-06-24 13:55:29,2024-06-24 15:24:29,880,342,Jamesstad,Meyerborough
26248,4366266646,15.0,1000,2020-01-28,3140.46,4366266646,2024-06-15 11:53:47,2024-06-15 13:39:47,576,149,South Michael,Hayesfort


In [26]:
# Drop the duplicate rows from the DataFrame
zugfahrten_df = zugfahrten_df.drop_duplicates()

# Display the DataFrame after dropping duplicate rows
print("DataFrame after dropping duplicate rows:")
print(zugfahrten_df)
zugfahrten_df.to_csv('droprows.csv', index=False)

DataFrame after dropping duplicate rows:
      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008           NaN  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id         Abfahrtszeit         Ankunftszeit Entfernung

In [27]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_fahrzeugnummer = zugfahrten_df[zugfahrten_df['Fahrzeugnummer'].isnull()]

print("Rows where 'Fahrzeugnummer' has no values before conversion:")
display(rows_with_missing_fahrzeugnummer)

Rows where 'Fahrzeugnummer' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
45,6944669989,NaN,1001,2020-09-18,3045.05,6944669989,2024-06-22 06:26:19,2024-06-22 10:22:19,493,95,North Stephaniemouth,East Eric
840,8237764087,NaN,1007,2022-10-02,1513.69,8237764087,2024-06-06 11:37:05,2024-06-06 15:13:05,918,105,North Alexandertown,Port Deanna
871,8695358391,NaN,1005,2023-02-03,2146.63,8695358391,2024-06-06 09:05:46,2024-06-06 11:27:46,690,350,West Alisonview,Port Anthony
2699,1262150578,NaN,1002,2024-01-05,1983.88,1262150578,2024-06-11 13:47:23,2024-06-11 17:31:23,498,428,Hortonport,South Victor
3016,1416452424,NaN,1004,2023-10-07,2577.26,1416452424,2024-06-04 00:11:12,2024-06-04 01:59:12,787,474,Michaelhaven,Port Nicholastown
6305,8477918761,NaN,1005,2022-03-27,4572.19,8477918761,2024-06-18 06:36:59,2024-06-18 11:33:59,316,469,North Cristina,Lake Michellehaven
6912,3899770054,NaN,1003,2022-03-25,3747.90,3899770054,2024-05-27 18:06:59,2024-05-27 21:21:59,318,446,East Pamelastad,East Kristyfurt
7576,0904570562,NaN,1006,2024-03-16,4259.26,0904570562,2024-06-16 06:27:21,2024-06-16 07:53:21,675,125,Mcdonaldmouth,Christopherport
7598,1843927334,NaN,1009,2023-07-09,2949.25,1843927334,2024-05-30 11:52:29,2024-05-30 16:19:29,710,338,Danielletown,Kevinhaven
7625,8953926787,NaN,1008,2021-01-22,990.81,8953926787,2024-06-10 22:43:53,2024-06-11 02:04:53,536,74,Rosalesshire,East Jeffrey


In [28]:
# Ensure the Fahrzeugnummer column is treated as numeric values
zugfahrten_df['Fahrzeugnummer'] = pd.to_numeric(zugfahrten_df['Fahrzeugnummer'], errors='coerce')

# Fill missing values in 'Fahrzeugnummer'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Fahrzeugnummer')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Fahrzeugnummer')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Fahrzeugnummer')] + 1
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Fahrzeugnummer')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Fahrzeugnummer')] - 1

# Display the DataFrame
zugfahrten_df.to_csv('rows.csv', index=False)

C:\Users\SEA\AppData\Local\Temp\ipykernel_17320\2973742870.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zugfahrten_df['Fahrzeugnummer'] = pd.to_numeric(zugfahrten_df['Fahrzeugnummer'], errors='coerce')


In [29]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25148 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fahrtnummer      25148 non-null  object 
 1   Fahrzeugnummer   25143 non-null  float64
 2   Buchungskreis    25107 non-null  object 
 3   Abfahrtsdatum    25114 non-null  object 
 4   Messung          25106 non-null  float64
 5   Zug id           25148 non-null  object 
 6   Abfahrtszeit     25096 non-null  object 
 7   Ankunftszeit     25116 non-null  object 
 8   Entfernung km    25099 non-null  object 
 9   Passagieranzahl  25110 non-null  object 
 10  Abfahrtsbahnhof  25113 non-null  object 
 11  Ankunftsbahnhof  25103 non-null  object 
dtypes: float64(2), object(10)
memory usage: 2.5+ MB


In [30]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_fahrzeugnummer = zugfahrten_df[zugfahrten_df['Fahrzeugnummer'].isnull()]

print("Rows where 'Fahrzeugnummer' has no values before conversion:")
display(rows_with_missing_fahrzeugnummer)

Rows where 'Fahrzeugnummer' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25225,4014906804,NaN,1004,2023-04-04,3280.63,4014906804,2024-06-21 18:46:57,2024-06-21 23:14:57,651,155,Lake Aaronberg,Santiagofurt
25306,8672585635,NaN,1008,2023-05-12,2735.81,8672585635,2024-06-02 08:24:52,2024-06-02 12:34:52,298,212,New Brett,Catherineberg
25541,1270865819,NaN,1008,2021-11-26,4424.59,1270865819,2024-06-06 03:23:19,2024-06-06 06:46:19,818,80,West Glennfort,Hoffmanfort
25833,4305878231,NaN,1000,2023-12-01,1252.93,4305878231,2024-06-03 10:17:40,2024-06-03 15:08:40,456,477,Taylorchester,Theodorechester
25973,8248983345,NaN,1000,2024-05-17,4225.10,8248983345,2024-06-25 16:30:52,2024-06-25 17:07:52,260,486,Bankstown,Hallport


### Buchungskreis 

In [31]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_buchungskreis = zugfahrten_df[zugfahrten_df['Buchungskreis'].isnull()]

print("Rows where 'Buchungskreis' has no values before conversion:")
display(rows_with_missing_buchungskreis)

Rows where 'Buchungskreis' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
59,2239050646,8.0,NaN,2022-02-10,1301.80,2239050646,2024-05-29 19:54:20,2024-05-29 21:33:20,538,473,Catherinechester,Collinschester
450,1065125257,10.0,NaN,2020-05-04,1979.80,1065125257,2024-06-25 13:35:13,2024-06-25 16:01:13,841,76,Morrismouth,Kathleenfurt
1031,4495325958,13.0,NaN,2021-06-01,4485.74,4495325958,2024-06-11 06:09:08,2024-06-11 10:45:08,215,483,Gayburgh,Elizabethview
1118,1474134785,1.0,NaN,2022-12-26,1180.10,1474134785,2024-06-02 17:31:56,2024-06-02 18:38:56,677,76,New Davidville,New Shannonborough
2608,0714499127,3.0,NaN,2021-09-01,990.05,0714499127,2024-06-07 18:07:33,2024-06-07 19:10:33,501,421,Mortonfort,Cantrellville
3123,1293484345,5.0,NaN,2022-05-26,2246.60,1293484345,2024-06-05 06:04:30,2024-06-05 07:56:30,135,314,Heatherburgh,Parkermouth
3564,6373607135,13.0,NaN,2021-12-11,1256.31,6373607135,2024-06-08 08:07:57,2024-06-08 12:34:57,901,492,Thomaston,Martinezfurt
4942,3690526426,1.0,NaN,2022-04-05,3855.77,3690526426,2024-06-15 03:29:52,2024-06-15 07:23:52,93,169,New Christina,Lake Lori
5538,3391611439,2.0,NaN,2021-07-28,3753.67,3391611439,2024-06-17 15:07:35,2024-06-17 16:46:35,679,73,East Larry,North Christopher
6793,6103726842,15.0,NaN,2023-07-20,4316.07,6103726842,2024-06-06 05:27:17,2024-06-06 07:58:17,195,300,West Allisonchester,North Seanberg


In [32]:
# Fill missing values in 'Buchungskreis'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Buchungskreis')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Buchungskreis')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Buchungskreis')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Buchungskreis')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Buchungskreis')]

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('kreis.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008           NaN  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id         Abfahrtszeit         Ankunftszeit Entfernung km  \
0      3094127253  2024-06-11 23:3

In [33]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_buchungskreis = zugfahrten_df[zugfahrten_df['Buchungskreis'].isnull()]

print("Rows where 'Buchungskreis' has no values before conversion:")
display(rows_with_missing_buchungskreis)

Rows where 'Buchungskreis' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25979,1535437009,3.0,NaN,2023-04-04,4554.83,1535437009,2024-06-22 09:11:08,2024-06-22 11:00:08,142,170,Luishaven,Alexanderside
26015,0656447169,1.0,NaN,2021-03-04,542.82,0656447169,2024-06-09 10:35:05,2024-06-09 13:38:05,304,394,West Amandachester,New Ann
26097,4653224538,5.0,NaN,2022-11-11,2017.93,4653224538,2024-05-30 01:56:14,2024-05-30 02:54:14,953,76,Joshuamouth,Osborneborough
26147,4694244410,11.0,NaN,2023-12-01,1381.83,4694244410,2024-06-15 12:21:57,2024-06-15 16:00:57,751,303,Timothytown,Jeffstad
26205,7942805615,3.0,NaN,2023-09-29,3635.70,7942805615,2024-06-01 05:55:41,2024-06-01 09:28:41,995,120,New Lindamouth,West Josephhaven


In [34]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum', 'Messung',
       'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum      34
Messung            42
Zug id              0
Abfahrtszeit       52
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


### Abfahrtsdatum

In [35]:
# Get unique values of 'Abfahrtsdatum'
unique_abfahrtsdatum = zugfahrten_df['Abfahrtsdatum'].unique()

# Display the unique values
print(unique_abfahrtsdatum)


['2020-12-29' '23/01/1999' '2021-03-14' ... '08/05/1970' '16/10/1988'
 '09/09/2011']


In [36]:
# Assuming zugfahrten_df is already defined and contains the 'Abfahrtsdatum' column

# Convert 'Abfahrtsdatum' to datetime, handling multiple formats
zugfahrten_df['Abfahrtsdatum'] = pd.to_datetime(zugfahrten_df['Abfahrtsdatum'], errors='coerce', dayfirst=True)

# Convert datetime to the desired format 'YYYY-MM-DD'
zugfahrten_df['Abfahrtsdatum'] = zugfahrten_df['Abfahrtsdatum'].dt.strftime('%Y-%m-%d')

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('datum.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008           NaN  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id         Abfahrtszeit         Ankunftszeit Entfernung km  \
0      3094127253  2024-06-11 23:3

C:\Users\SEA\AppData\Local\Temp\ipykernel_17320\1551397323.py:4: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  zugfahrten_df['Abfahrtsdatum'] = pd.to_datetime(zugfahrten_df['Abfahrtsdatum'], errors='coerce', dayfirst=True)
C:\Users\SEA\AppData\Local\Temp\ipykernel_17320\1551397323.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zugfahrten_df['Abfahrtsdatum'] = pd.to_datetime(zugfahrten_df['Abfahrtsdatum'], errors='coerce', dayfirst=True)
C:\Users\SEA\AppData\Local\Temp\ipykernel_17320\1551397323.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

In [37]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtsdatum', 'Messung',
       'Abfahrtszeit', 'Ankunftszeit', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer          0
Fahrzeugnummer       5
Buchungskreis        5
Abfahrtsdatum      624
Messung             42
Zug id               0
Abfahrtszeit        52
Ankunftszeit        32
Entfernung km       49
Passagieranzahl     38
Abfahrtsbahnhof     35
Ankunftsbahnhof     45
dtype: int64


In [38]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_abfahrtsdatum = zugfahrten_df[zugfahrten_df['Abfahrtsdatum'].isnull()]

print("Rows where 'Abfahrtsdatum' has no values before conversion:")
display(rows_with_missing_abfahrtsdatum)

Rows where 'Abfahrtsdatum' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
8,8322069260,1.0,1001,NaN,1893.15,8322069260,2024-06-13 07:45:16,2024-06-13 09:38:16,531,190,Davidview,Millermouth
10,8322069260,3.0,1001,NaN,4914.01,8322069260,2024-06-13 07:45:16,2024-06-13 09:38:16,531,190,Davidview,Millermouth
112,0939272042,17.0,1000,NaN,4192.03,0939272042,2024-06-20 11:10:28,2024-06-20 15:14:28,942,51,New Stevenstad,Smithshire
129,4079331200,10.0,XXXX,NaN,1462.96,4079331200,2024-06-08 02:28:47,2024-06-08 05:04:47,662,173,Tiffanybury,East Steventown
184,0739456991,8.0,1002,NaN,3915.08,0739456991,2024-06-08 13:20:24,2024-06-08 17:45:24,755,183,East Walterbury,North Craigbury
...,...,...,...,...,...,...,...,...,...,...,...,...
26066,4517874279,6.0,1009,NaN,1749.77,4517874279,2024-06-12 10:15:30,2024-06-12 13:30:30,971,74,Castroview,Dominguezstad
26106,8078847045,4.0,1006,NaN,1027.39,8078847045,2024-06-18 02:12:00,2024-06-18 06:46:00,167,92,North Danielland,Port Jessica
26137,9232618661,5.0,1009,NaN,4541.94,9232618661,2024-06-11 13:06:10,2024-06-11 14:29:10,637,215,West Javierchester,New Meredithborough
26164,9917917157,3.0,1002,NaN,884.54,9917917157,2024-06-22 20:29:04,2024-06-22 23:42:04,307,87,Danielview,North Brandonmouth


In [39]:
# Fill missing values in 'Abfahrtsdatum'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsdatum')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsdatum')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Abfahrtsdatum')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsdatum')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Abfahrtsdatum')]

# Display the DataFrame
zugfahrten_df.to_csv('abfahrtsdatum.csv', index=False)

In [40]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_abfahrtsdatum = zugfahrten_df[zugfahrten_df['Abfahrtsdatum'].isnull()]

print("Rows where 'Abfahrtsdatum' has no values before conversion:")
display(rows_with_missing_abfahrtsdatum)

Rows where 'Abfahrtsdatum' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
11778,8111468295,1.0,XXXX,NaN,4555.12,8111468295,2024-06-18 04:59:50,2024-06-18 08:37:50,245,77,Mccoymouth,Lake Christopherchester
11779,8111468295,2.0,XXXX,NaN,1096.97,8111468295,2024-06-18 04:59:50,2024-06-18 08:37:50,245,77,Mccoymouth,Lake Christopherchester
23280,4748497734,1.0,XXXX,NaN,940.22,4748497734,2024-06-15 09:17:18,2024-06-15 10:57:18,193,454,South Matthew,Mendozaburgh
23281,4748497734,2.0,XXXX,NaN,2606.45,4748497734,2024-06-15 09:17:18,2024-06-15 10:57:18,193,454,South Matthew,Mendozaburgh
25045,1892773720,1.0,1005,NaN,2359.23,1892773720,2024-06-04 11:46:08,2024-06-04 12:20:08,80,139,New Debraburgh,Pruittfort
25046,8496552105,18.0,1000,NaN,3087.18,8496552105,2024-06-22 18:30:52,2024-06-22 22:00:52,372,404,Rossberg,Roseside
25061,8477918761,13.0,1005,NaN,4456.69,8477918761,2024-06-18 06:36:59,2024-06-18 11:33:59,316,469,North Cristina,Lake Michellehaven
25094,0967342393,10.0,1008,NaN,4524.53,0967342393,2024-05-30 04:40:16,2024-05-30 07:20:16,734,277,New Sonyatown,Lake Kylefort
25153,2639357392,3.0,1007,NaN,3694.14,2639357392,2024-06-11 08:23:17,2024-06-11 12:26:17,134,152,Lake Debraview,North Billy
25157,2085545360,4.0,XXXX,NaN,1036.86,2085545360,2024-06-02 07:42:12,2024-06-02 11:21:12,798,290,Port Meagan,East Jo


In [41]:
# Fill missing values in 'Abfahrtsdatum'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsdatum')]):
        same_zug_id_rows = zugfahrten_df[zugfahrten_df['Zug id'] == zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')]]
        known_dates = same_zug_id_rows['Abfahrtsdatum'].dropna().unique()
        if known_dates.size > 0:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsdatum')] = known_dates[0]

# Display the DataFrame
zugfahrten_df.to_csv('right_date.csv', index=False)

In [42]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25148 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Fahrtnummer      25148 non-null  object 
 1   Fahrzeugnummer   25143 non-null  float64
 2   Buchungskreis    25143 non-null  object 
 3   Abfahrtsdatum    25148 non-null  object 
 4   Messung          25106 non-null  float64
 5   Zug id           25148 non-null  object 
 6   Abfahrtszeit     25096 non-null  object 
 7   Ankunftszeit     25116 non-null  object 
 8   Entfernung km    25099 non-null  object 
 9   Passagieranzahl  25110 non-null  object 
 10  Abfahrtsbahnhof  25113 non-null  object 
 11  Ankunftsbahnhof  25103 non-null  object 
dtypes: float64(2), object(10)
memory usage: 2.5+ MB


In [43]:

# Assuming zugfahrten_df is already defined and contains the 'Abfahrtsdatum' column

# Convert 'Abfahrtsdatum' to datetime and ensure the format is 'YYYY-MM-DD'
zugfahrten_df['Abfahrtsdatum'] = pd.to_datetime(zugfahrten_df['Abfahrtsdatum'], format='%Y-%m-%d', errors='coerce')

# Display the DataFrame to verify the changes
print(zugfahrten_df)


      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id         Abfahrtszeit         Ankunftszeit Entfernung km  \
0      3094127253  2024-06-11 23:3

C:\Users\SEA\AppData\Local\Temp\ipykernel_17320\3920231494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zugfahrten_df['Abfahrtsdatum'] = pd.to_datetime(zugfahrten_df['Abfahrtsdatum'], format='%Y-%m-%d', errors='coerce')


In [44]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25148 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25148 non-null  object        
 1   Fahrzeugnummer   25143 non-null  float64       
 2   Buchungskreis    25143 non-null  object        
 3   Abfahrtsdatum    25148 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25148 non-null  object        
 6   Abfahrtszeit     25096 non-null  object        
 7   Ankunftszeit     25116 non-null  object        
 8   Entfernung km    25099 non-null  object        
 9   Passagieranzahl  25110 non-null  object        
 10  Abfahrtsbahnhof  25113 non-null  object        
 11  Ankunftsbahnhof  25103 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 2.5+ MB


In [45]:
min_abfahrtsdatum = zugfahrten_df['Abfahrtsdatum'].min()
max_abfahrtsdatum = zugfahrten_df['Abfahrtsdatum'].max()

# Print the results
print("Minimum Abfahrtsdatum:", min_abfahrtsdatum)
print("Maximum Abfahrtsdatum:", max_abfahrtsdatum)

Minimum Abfahrtsdatum: 2020-01-01 00:00:00
Maximum Abfahrtsdatum: 2024-06-24 00:00:00


### Messung

In [46]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_messung = zugfahrten_df[zugfahrten_df['Messung'].isnull()]

print("Rows where 'Messung' has no values before conversion:")
display(rows_with_missing_messung)

Rows where 'Messung' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
96,0939272042,1.0,1000,2020-06-15,NaN,0939272042,2024-06-20 11:10:28,2024-06-20 15:14:28,942,51,New Stevenstad,Smithshire
727,9417400399,13.0,1008,2020-02-12,NaN,9417400399,2024-06-15 08:14:42,2024-06-15 08:58:42,829,111,Port Jaredberg,Port Lauren
929,2150031639,3.0,1007,2020-12-03,NaN,2150031639,2024-06-09 05:11:25,2024-06-09 09:20:25,115,290,Connertown,South Danielland
3619,4116681925,2.0,1001,2021-04-28,NaN,4116681925,2024-06-07 22:02:24,2024-06-08 01:37:24,183,71,Williamsmouth,New Nicholasshire
4142,8577318898,15.0,1004,2022-11-27,NaN,8577318898,2024-06-14 17:46:24,performance,414,150,Allenland,East Angelbury
4946,3690526426,5.0,1004,2022-04-05,NaN,3690526426,2024-06-15 03:29:52,2024-06-15 07:23:52,93,169,New Christina,Lake Lori
5622,0899934143,9.0,1007,2023-08-05,NaN,0899934143,2024-06-22 23:44:19,2024-06-23 01:18:19,441,95,North Frank,Sharonburgh
6908,3899770054,6.0,1003,2022-03-25,NaN,3899770054,2024-05-27 18:06:59,2024-05-27 21:21:59,318,446,East Pamelastad,East Kristyfurt
7509,9203259262,9.0,1002,2020-03-09,NaN,9203259262,2024-06-20 12:00:46,2024-06-20 14:45:46,764,400,Rothbury,Martinport
7983,1091609511,10.0,1007,2020-12-30,NaN,1091609511,2024-06-01 12:01:15,2024-06-01 14:52:15,730,400,Thompsonberg,Port Keith


In [47]:
# Drop rows where 'Messung' is missing
zugfahrten_df = zugfahrten_df.dropna(subset=['Messung'])

# Display the DataFrame to verify the changes
print(zugfahrten_df)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id         Abfahrtszeit         Ankunftszeit Entfernung km  \
0      3094127253  2024-06-11 23:3

In [48]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25054 non-null  object        
 7   Ankunftszeit     25074 non-null  object        
 8   Entfernung km    25057 non-null  object        
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](1), float64(2), object(9)
memory usage: 2.5+ MB


In [49]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtszeit', 'Ankunftszeit',
       'Entfernung km', 'Passagieranzahl', 'Abfahrtsbahnhof',
       'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit       52
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


### Abfahrtszeit       

In [50]:
# Get unique values of 'Abfahrtsdatum'
unique_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].unique()

# Display the unique values
print(unique_abfahrtszeit)

['2024-06-11 23:31:02' '2024-06-13 07:45:16' '2024-06-08 10:40:11' ...
 'security' 'relationship' 'somebody']


In [51]:
# Convert 'Abfahrtszeit' to datetime, coercing errors
zugfahrten_df['Abfahrtszeit'] = pd.to_datetime(zugfahrten_df['Abfahrtszeit'], errors='coerce')

# Drop rows where 'Abfahrtszeit' could not be converted to a valid datetime

# Display the cleaned DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('zeit.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit Entfernung km  \
0      3094127253 2024-06-11 23:31:

In [52]:
# Get unique values of 'Abfahrtsdatum'
unique_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].unique()

# Display the unique values
print(unique_abfahrtszeit)

<DatetimeArray>
['2024-06-11 23:31:02', '2024-06-13 07:45:16', '2024-06-08 10:40:11',
 '2024-06-22 06:26:19', '2024-05-29 19:54:20', '2024-05-27 05:24:14',
 '2024-05-30 02:11:16', '2024-06-22 08:50:13',                 'NaT',
 '2024-06-03 20:27:19',
 ...
 '2024-05-31 16:35:15', '2024-06-09 03:04:15', '2024-06-20 03:21:24',
 '2024-06-06 07:44:06', '2024-06-22 14:55:05', '2024-06-04 02:33:09',
 '2024-06-03 08:05:01', '2024-06-13 14:51:42', '2024-06-24 10:28:45',
 '2024-06-23 19:14:24']
Length: 2277, dtype: datetime64[ns]


In [53]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Abfahrtszeit', 'Ankunftszeit',
       'Entfernung km', 'Passagieranzahl', 'Abfahrtsbahnhof',
       'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer          0
Fahrzeugnummer       5
Buchungskreis        5
Abfahrtsdatum        0
Messung              0
Zug id               0
Abfahrtszeit       142
Ankunftszeit        32
Entfernung km       49
Passagieranzahl     38
Abfahrtsbahnhof     35
Ankunftsbahnhof     45
dtype: int64


In [54]:
# Fill missing values in 'Abfahrtsdatum'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtszeit')]):
        same_zug_id_rows = zugfahrten_df[zugfahrten_df['Zug id'] == zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')]]
        known_dates = same_zug_id_rows['Abfahrtszeit'].dropna().unique()
        if known_dates.size > 0:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtszeit')] = known_dates[0]

# Display the DataFrame
zugfahrten_df.to_csv('Abfahrtszeit.csv', index=False)

In [55]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Ankunftszeit', 'Entfernung km',
       'Passagieranzahl', 'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit       32
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


In [56]:
min_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].min()
max_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].max()

# Print the results
print("Minimum Abfahrtszeit:", min_abfahrtszeit)
print("Maximum Abfahrtszeit:", max_abfahrtszeit)

Minimum Abfahrtszeit: 2024-05-26 23:28:38
Maximum Abfahrtszeit: 2024-06-25 23:18:44


In [57]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25074 non-null  object        
 8   Entfernung km    25057 non-null  object        
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 2.5+ MB


### Ankunftszeit     

In [58]:
# Get unique values of 'Ankunftszeit'
unique_ankunftszeit = zugfahrten_df['Ankunftszeit'].unique()

# Display the unique values
print(unique_ankunftszeit)

['2024-06-12 02:02:02' '2024-06-13 09:38:16' '2024-06-08 14:24:11' ...
 'fire' 'build' 'interesting']


In [59]:

# Convert 'Abfahrtszeit' to datetime, coercing errors
zugfahrten_df['Ankunftszeit'] = pd.to_datetime(zugfahrten_df['Ankunftszeit'], errors='coerce')


# Format 'Abfahrtszeit' to 'YYYY-MM-DD HH:MM:SS'
zugfahrten_df['Ankunftszeit'] = zugfahrten_df['Ankunftszeit'].dt.strftime('%Y-%m-%d %H:%M:%S')
zugfahrten_df.to_csv('Ankunftszeit.csv', index=False)


In [60]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     24965 non-null  object        
 8   Entfernung km    25057 non-null  object        
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 2.5+ MB


In [61]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Ankunftszeit', 'Entfernung km',
       'Passagieranzahl', 'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer          0
Fahrzeugnummer       5
Buchungskreis        5
Abfahrtsdatum        0
Messung              0
Zug id               0
Abfahrtszeit         0
Ankunftszeit       141
Entfernung km       49
Passagieranzahl     38
Abfahrtsbahnhof     35
Ankunftsbahnhof     45
dtype: int64


In [62]:
# Fill missing values in 'Abfahrtsdatum'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Ankunftszeit')]):
        same_zug_id_rows = zugfahrten_df[zugfahrten_df['Zug id'] == zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')]]
        known_dates = same_zug_id_rows['Ankunftszeit'].dropna().unique()
        if known_dates.size > 0:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Ankunftszeit')] = known_dates[0]

# Display the DataFrame
zugfahrten_df.to_csv('RealAnkunftszeit.csv', index=False)

In [63]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit        0
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


In [64]:
min_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].min()
max_abfahrtszeit = zugfahrten_df['Abfahrtszeit'].max()

# Print the results
print("Minimum Abfahrtszeit:", min_abfahrtszeit)
print("Maximum Abfahrtszeit:", max_abfahrtszeit)

Minimum Abfahrtszeit: 2024-05-26 23:28:38
Maximum Abfahrtszeit: 2024-06-25 23:18:44


In [65]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25057 non-null  object        
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 2.5+ MB


In [66]:
# Filter rows where 'Abfahrtszeit' is not earlier than 'Ankunftszeit'
false_rows = zugfahrten_df[zugfahrten_df['Abfahrtszeit'] >= zugfahrten_df['Ankunftszeit']]

# Display the rows where 'Abfahrtszeit' is not earlier than 'Ankunftszeit'
print(false_rows)

Empty DataFrame
Columns: [Fahrtnummer, Fahrzeugnummer, Buchungskreis, Abfahrtsdatum, Messung, Zug id, Abfahrtszeit, Ankunftszeit, Entfernung km, Passagieranzahl, Abfahrtsbahnhof, Ankunftsbahnhof]
Index: []


In [67]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25057 non-null  object        
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(2), object(8)
memory usage: 2.5+ MB


In [68]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit        0
Entfernung km      49
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


### Entfernung km

In [69]:
# Get unique values of 'Ankunftszeit'
unique_entfernung = zugfahrten_df['Entfernung km'].unique()

# Display the unique values
print(unique_entfernung)

['688' '531' '624' '493' '538' '464' '578' '459' '495' '942' '637' '662'
 '132' '906' '755' nan '918' '958' '800' '354' '582' '481' '545' '254'
 '92' '398' '440' '744' '100' '853' '677' '319' '841' '287' '980' '919'
 '880' '608' '740' '144' '231' '235' '933' '691' '650' '211' '439' '914'
 '67' '861' '426' '473' '62' 'smile' '928' '569' '962' '185' '829' '748'
 '912' '299' '683' '660' '97' '947' '438' '944' '322' '364' '690' '782'
 '524' '230' '115' '117' '142' '256' '629' '542' '659' '586' '215' '345'
 '469' '384' '145' '571' '836' '270' '366' '898' '284' '435' '726' '816'
 '521' '68' '600' '840' '642' '534' '576' '612' '785' '911' '625' '597'
 '489' '870' '447' '990' '251' '157' '653' '401' 'show' '71' '51' '172'
 '240' '110' '336' '893' '930' '207' '421' '80' '956' '544' '805' '378'
 '982' '360' '596' '467' '985' '878' '88' '134' '988' '292' '719' '347'
 '487' '983' '160' '831' '894' '736' '808' '814' '602' '410' '689' '617'
 '810' '648' '131' '408' 'begin' '355' '762' '888' '484' '3

In [70]:
# Replace non-numeric values with NaN
zugfahrten_df['Entfernung km'] = pd.to_numeric(zugfahrten_df['Entfernung km'], errors='coerce')

# Display the DataFrame to verify the changes
display(zugfahrten_df)

,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
0,3094127253,1.0,1000,2020-12-29,2321.14,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688.0,470,Millerside,Lake Adam
1,3094127253,2.0,1000,2020-12-29,2361.35,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688.0,470,Millerside,Lake Adam
2,3094127253,3.0,1000,2020-12-29,3078.98,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688.0,470,Millerside,Lake Adam
3,3094127253,4.0,1000,2020-12-29,4063.26,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688.0,470,Millerside,Lake Adam
4,3094127253,5.0,1000,2020-12-29,624.47,3094127253,2024-06-11 23:31:02,2024-06-12 02:02:02,688.0,470,Millerside,Lake Adam
...,...,...,...,...,...,...,...,...,...,...,...,...
26211,0799554257,11.0,1008,2023-05-19,4205.58,0799554257,2024-06-15 22:31:26,2024-06-16 02:37:26,875.0,227,Tiffanymouth,Port Aliciaport
26212,4191950654,9.0,1000,2023-06-17,3872.10,4191950654,2024-06-13 13:15:21,2024-06-13 17:40:21,129.0,266,NaN,Mataton
26215,0117443352,3.0,1004,2023-08-01,4294.16,0117443352,2024-06-08 15:11:53,2024-06-08 18:39:53,627.0,414,South Jordanstad,Port Karen
26235,7772769811,1.0,1002,2022-10-22,3249.77,7772769811,2024-06-19 01:18:57,2024-06-19 06:10:57,484.0,304,Smithmouth,North Sarah


In [71]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_entfernung = zugfahrten_df[zugfahrten_df['Entfernung km'].isnull()]

print("Rows where 'Entfernung km' has no values before conversion:")
display(rows_with_entfernung)

Rows where 'Entfernung km' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
180,0739456991,4.0,1002,2023-09-23,1833.13,0739456991,2024-06-08 13:20:24,2024-06-08 17:45:24,NaN,183,East Walterbury,North Craigbury
656,7868722371,4.0,1004,2020-08-24,4491.53,7868722371,2024-06-02 14:38:00,2024-06-02 15:58:00,NaN,50,Thomasshire,Stanleyburgh
1457,7512879927,8.0,1007,2022-06-15,4500.71,7512879927,2024-05-27 05:40:33,2024-05-27 09:33:33,NaN,292,Isaacland,South Robertmouth
2043,0805179586,5.0,1009,2022-07-10,1157.11,0805179586,2024-06-02 08:16:17,2024-06-02 10:40:17,NaN,143,Carlsonchester,Williamsview
2065,9396096446,9.0,1005,2022-08-28,2373.35,9396096446,2024-06-06 10:16:23,2024-06-06 14:43:23,NaN,55,North Thomasberg,North Victoriaberg
...,...,...,...,...,...,...,...,...,...,...,...,...
25380,1388934800,12.0,1003,2024-04-06,2929.03,1388934800,2024-06-13 17:32:02,2024-06-13 18:51:02,NaN,441,Brittanytown,Wrightchester
25447,3896788145,5.0,1007,2023-05-15,913.05,3896788145,2024-06-10 11:27:06,2024-06-10 12:50:06,NaN,91,West Savannahport,Mooretown
25493,1519602640,7.0,1007,2020-01-05,3057.13,1519602640,2024-05-27 05:33:56,2024-05-27 06:26:56,NaN,356,North Theresa,Port Davidside
25573,3949411792,3.0,1005,2024-03-04,1094.30,3949411792,2024-06-14 06:56:31,2024-06-14 11:23:31,NaN,443,South Kyle,Stephenville


In [72]:
# Fill missing values in 'Buchungskreis'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Entfernung km')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Entfernung km')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Entfernung km')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Entfernung km')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Entfernung km')]

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('Entfernungkm.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [73]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_entfernung = zugfahrten_df[zugfahrten_df['Entfernung km'].isnull()]

print("Rows where 'Entfernung km' has no values before conversion:")
display(rows_with_entfernung)

Rows where 'Entfernung km' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25089,1929538854,13.0,1007,2022-07-29,4566.76,1929538854,2024-06-09 20:49:51,2024-06-09 23:39:51,NaN,250,Cochranside,South Rachel
25190,9935879448,15.0,1000,2020-06-27,4611.10,9935879448,2024-05-27 22:59:07,2024-05-28 00:55:07,NaN,124,South Conniestad,Port Arthurview
25380,1388934800,12.0,1003,2024-04-06,2929.03,1388934800,2024-06-13 17:32:02,2024-06-13 18:51:02,NaN,441,Brittanytown,Wrightchester
25447,3896788145,5.0,1007,2023-05-15,913.05,3896788145,2024-06-10 11:27:06,2024-06-10 12:50:06,NaN,91,West Savannahport,Mooretown
25493,1519602640,7.0,1007,2020-01-05,3057.13,1519602640,2024-05-27 05:33:56,2024-05-27 06:26:56,NaN,356,North Theresa,Port Davidside
25573,3949411792,3.0,1005,2024-03-04,1094.30,3949411792,2024-06-14 06:56:31,2024-06-14 11:23:31,NaN,443,South Kyle,Stephenville
26240,7705851007,3.0,1009,2020-05-20,969.09,7705851007,2024-06-20 05:17:37,2024-06-20 10:01:37,NaN,358,South Craighaven,Cummingsfort


In [74]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit        0
Entfernung km       7
Passagieranzahl    38
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


In [75]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25099 non-null  float64       
 9   Passagieranzahl  25068 non-null  object        
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(3), object(7)
memory usage: 2.5+ MB


In [76]:
min_entfernung = zugfahrten_df['Entfernung km'].min()
max_entfernung = zugfahrten_df['Entfernung km'].max()

# Print the results
print("Minimum Entfernung km:", min_entfernung)
print("Maximum Entfernung km:", max_entfernung)

Minimum Entfernung km: 50.0
Maximum Entfernung km: 1000.0


### Passagieranzahl    

In [77]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_Passagieranzahl = zugfahrten_df[zugfahrten_df['Passagieranzahl'].isnull()]

print("Rows where 'Messung' has no values before conversion:")
display(rows_with_missing_Passagieranzahl)

Rows where 'Messung' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
2391,9777939530,15.0,1008,2020-06-19,4000.63,9777939530,2024-06-08 21:27:13,2024-06-08 23:35:13,108.0,NaN,East Kylehaven,North Christinashire
3104,5249158181,4.0,1005,2020-03-07,4209.47,5249158181,2024-06-19 23:45:53,2024-06-20 02:06:53,413.0,NaN,West Laurafurt,Scottside
3155,1696083073,12.0,1006,2022-11-01,3696.59,1696083073,2024-06-13 21:53:04,2024-06-14 02:27:04,248.0,NaN,North Jennifer,Cindychester
4276,3041826626,19.0,1002,2023-09-22,4191.76,3041826626,2024-06-18 15:55:32,2024-06-18 19:24:32,100.0,NaN,South Pedro,Ericaland
4786,0873473026,5.0,1006,2023-10-25,3535.76,0873473026,2024-06-23 22:33:20,2024-06-24 03:32:20,723.0,NaN,South Josephview,North Robertview
7429,1079932856,18.0,1001,2020-07-14,4116.58,1079932856,2024-06-21 12:31:49,2024-06-21 14:21:49,402.0,NaN,Jacquelineton,Mcdonaldstad
7829,4379612572,6.0,1005,2023-12-02,1455.67,4379612572,2024-06-10 17:21:40,2024-06-10 19:51:40,685.0,NaN,New James,Port Peterfurt
8290,8276546416,11.0,1002,2020-02-28,3325.09,8276546416,2024-06-07 01:21:53,2024-06-07 03:54:53,699.0,NaN,North Jamie,South Jessica
8737,0248001571,7.0,1002,2023-11-30,639.80,0248001571,2024-06-15 14:46:35,2024-06-15 18:38:35,645.0,NaN,East Robin,West Robertbury
9319,2088205650,14.0,1005,2022-06-02,1650.78,2088205650,2024-06-14 17:51:09,2024-06-14 22:37:09,120.0,NaN,Colemanborough,Calvinshire


In [78]:
# Get unique values of 'Ankunftszeit'
unique_entfernung = zugfahrten_df['Passagieranzahl'].unique()

# Display the unique values
print(unique_entfernung)

['470' '190' '77' '95' '473' '324' '268' '442' '82' '51' '89' '173' '132'
 '368' '183' '247' '50' '441' '451' 'important' '102' '372' 'still' '318'
 'audience' '366' '70' '397' '316' '385' '347' '221' '180' '336' '88' '76'
 '370' '140' '479' '342' '439' '325' '293' '462' '418' '311' '433' '400'
 '493' '93' 'ball' '382' '161' '124' '236' '256' '234' '211' '165' '111'
 '289' '71' '437' 'collection' '412' '307' '169' '223' '83' '411' '105'
 '259' '350' '308' '118' '340' '290' '319' '123' '364' '128' '79' '100'
 '483' '302' '327' '227' '182' '164' '485' '146' '468' '297' '393' '196'
 '475' '337' '217' '496' '239' '436' 'policy' '74' '152' '374' '264' '125'
 '484' '425' '396' '230' 'last' '292' '481' '384' '453' '75' '97' '249'
 '141' '330' '309' '346' '287' '489' '224' '189' '250' '492' '378' '335'
 '275' 'instead' '300' '381' '263' 'edge' '109' '298' '345' '193' '131'
 '495' '139' '359' '315' '399' '225' '499' '261' '208' '170' '143' 'sport'
 '55' '98' '73' '304' '394' '69' '137' '267' '4

In [79]:
# Replace non-numeric values with NaN
zugfahrten_df['Passagieranzahl'] = pd.to_numeric(zugfahrten_df['Passagieranzahl'], errors='coerce')

# Display the DataFrame to verify the changes
print(zugfahrten_df)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [80]:
# Get unique values of 'Ankunftszeit'
unique_entfernung = zugfahrten_df['Passagieranzahl'].unique()

# Display the unique values
print(unique_entfernung)

[470. 190.  77.  95. 473. 324. 268. 442.  82.  51.  89. 173. 132. 368.
 183. 247.  50. 441. 451.  nan 102. 372. 318. 366.  70. 397. 316. 385.
 347. 221. 180. 336.  88.  76. 370. 140. 479. 342. 439. 325. 293. 462.
 418. 311. 433. 400. 493.  93. 382. 161. 124. 236. 256. 234. 211. 165.
 111. 289.  71. 437. 412. 307. 169. 223.  83. 411. 105. 259. 350. 308.
 118. 340. 290. 319. 123. 364. 128.  79. 100. 483. 302. 327. 227. 182.
 164. 485. 146. 468. 297. 393. 196. 475. 337. 217. 496. 239. 436.  74.
 152. 374. 264. 125. 484. 425. 396. 230. 292. 481. 384. 453.  75.  97.
 249. 141. 330. 309. 346. 287. 489. 224. 189. 250. 492. 378. 335. 275.
 300. 381. 263. 109. 298. 345. 193. 131. 495. 139. 359. 315. 399. 225.
 499. 261. 208. 170. 143.  55.  98.  73. 304. 394.  69. 137. 267. 421.
 215. 422. 231. 314. 155. 465. 280. 373. 467. 380. 328. 466. 389. 255.
 241. 295. 159. 158. 358. 463.  63. 444. 284. 303. 133. 226. 160. 417.
 265. 428. 212. 138. 488. 424. 233. 410. 379. 478. 416. 426. 449. 480.
 266. 

In [81]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_messung = zugfahrten_df[zugfahrten_df['Passagieranzahl'].isnull()]

print("Rows where 'Messung' has no values before conversion:")
display(rows_with_missing_messung)

Rows where 'Messung' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
234,1560100653,9.0,1000,2020-01-20,1972.46,1560100653,2024-06-18 01:19:45,2024-06-18 05:56:45,800.0,NaN,Andrewstad,Sherryfurt
276,3802510282,1.0,XXXX,2020-09-03,3661.83,3802510282,2024-06-25 12:08:37,2024-06-25 14:11:37,582.0,NaN,West Joshuafurt,Patriciashire
278,3802510282,3.0,XXXX,2020-09-03,2735.01,3802510282,2024-06-25 12:08:37,2024-06-25 14:11:37,582.0,NaN,West Joshuafurt,Patriciashire
585,3553892234,10.0,1007,2022-10-23,2979.46,3553892234,2024-06-01 16:45:05,2024-06-01 17:16:05,914.0,NaN,Lake Timothystad,South Tylerborough
768,4207933620,4.0,XXXX,2023-07-15,2639.52,4207933620,2024-06-08 09:16:45,2024-06-08 10:45:45,683.0,NaN,West Michelleland,South Garyshire
...,...,...,...,...,...,...,...,...,...,...,...,...
24864,3607663309,2.0,1001,2022-02-14,3025.13,3607663309,2024-05-30 05:17:16,2024-05-30 08:49:16,523.0,NaN,Lake Travisfurt,Port Joseview
25378,7937698187,5.0,XXXX,2020-12-01,2135.35,7937698187,2024-06-08 13:11:43,2024-06-08 15:40:43,857.0,NaN,Christophermouth,Port Nataliebury
25682,0202022090,6.0,1005,2020-05-09,2529.34,0202022090,2024-06-13 13:42:41,2024-06-13 16:23:41,713.0,NaN,South Christinestad,West Amber
25773,5115563759,8.0,1000,2022-01-05,1011.87,5115563759,2024-06-16 12:57:02,2024-06-16 15:43:02,901.0,NaN,Diazbury,Boydburgh


In [82]:
# Fill missing values in 'Buchungskreis'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Passagieranzahl')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Passagieranzahl')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Passagieranzahl')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Passagieranzahl')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Passagieranzahl')]

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('Passagieranzahl.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [83]:
min_Passagieranzahl = zugfahrten_df['Passagieranzahl'].min()
max_Passagieranzahl = zugfahrten_df['Passagieranzahl'].max()

# Print the results
print("Minimum Passagieranzahl km:", min_Passagieranzahl)
print("Maximum Passagieranzahl km:", max_Passagieranzahl)

Minimum Passagieranzahl km: 50.0
Maximum Passagieranzahl km: 500.0


In [84]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_messung = zugfahrten_df[zugfahrten_df['Passagieranzahl'].isnull()]

print("Rows where 'Messung' has no values before conversion:")
display(rows_with_missing_messung)

Rows where 'Messung' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25378,7937698187,5.0,XXXX,2020-12-01,2135.35,7937698187,2024-06-08 13:11:43,2024-06-08 15:40:43,857.0,NaN,Christophermouth,Port Nataliebury
25682,0202022090,6.0,1005,2020-05-09,2529.34,0202022090,2024-06-13 13:42:41,2024-06-13 16:23:41,713.0,NaN,South Christinestad,West Amber
25773,5115563759,8.0,1000,2022-01-05,1011.87,5115563759,2024-06-16 12:57:02,2024-06-16 15:43:02,901.0,NaN,Diazbury,Boydburgh
26132,2245341804,14.0,1008,2023-04-07,936.38,2245341804,2024-06-05 01:43:48,2024-06-05 04:01:48,89.0,NaN,Robinport,West Aprilshire


In [85]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25099 non-null  float64       
 9   Passagieranzahl  25102 non-null  float64       
 10  Abfahrtsbahnhof  25071 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(6)
memory usage: 2.5+ MB


In [86]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit        0
Entfernung km       7
Passagieranzahl     4
Abfahrtsbahnhof    35
Ankunftsbahnhof    45
dtype: int64


### Abfahrtsbahnhof    

In [87]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_Abfahrtsbahnhof = zugfahrten_df[zugfahrten_df['Abfahrtsbahnhof'].isnull()]

print("Rows where 'Abfahrtsbahnhof' has no values before conversion:")
display(rows_with_missing_Abfahrtsbahnhof)

Rows where 'Abfahrtsbahnhof' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
520,7740103208,3.0,1000,2022-07-08,4997.35,7740103208,2024-05-28 07:46:58,2024-05-28 09:50:58,231.0,462.0,NaN,West Williambury
945,6238971070,1.0,1000,2022-11-05,973.63,6238971070,2024-05-31 22:57:29,2024-06-01 01:08:29,256.0,123.0,NaN,Richardborough
1512,6330600335,3.0,1000,2022-04-03,2238.75,6330600335,2024-06-06 15:35:00,2024-06-06 20:16:00,110.0,97.0,NaN,Lake Paul
1689,7985315406,7.0,XXXX,2022-05-19,4430.47,7985315406,2024-06-08 02:11:29,2024-06-08 06:38:29,360.0,492.0,NaN,Petersonville
1775,9475477779,3.0,1000,2023-03-02,3407.15,9475477779,2024-06-13 17:33:26,2024-06-13 21:14:26,878.0,381.0,NaN,East Jeffrey
1847,5079147543,3.0,1004,2022-07-28,4498.24,5079147543,2024-06-12 05:45:43,2024-06-12 09:06:43,983.0,131.0,NaN,South Jessicashire
2110,6720119172,17.0,1008,2022-05-29,3658.69,6720119172,2024-06-02 04:05:31,2024-06-02 07:58:31,683.0,73.0,NaN,East Taraville
3574,4034126354,5.0,XXXX,2023-04-15,1199.14,4034126354,2024-05-29 00:44:20,2024-05-29 03:42:20,324.0,478.0,NaN,Tamarashire
5046,9347727632,3.0,1001,2021-11-27,3973.65,9347727632,2024-06-25 16:48:48,2024-06-25 19:54:48,721.0,68.0,NaN,Lake Megan
7116,3473313787,14.0,1002,2021-11-04,4293.62,3473313787,2024-06-11 02:44:03,2024-06-11 03:14:03,80.0,154.0,NaN,Holderton


In [88]:
# Get unique values of 'Ankunftszeit'
unique_entfernung = zugfahrten_df['Abfahrtsbahnhof'].unique()
print(unique_entfernung)


['Millerside' 'Davidview' 'Lake Mariaport' ... 'Patriciaborough'
 'New Jason' 'Travisland']


In [89]:
# Fill missing values in 'Buchungskreis'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsbahnhof')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsbahnhof')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Abfahrtsbahnhof')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Abfahrtsbahnhof')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Abfahrtsbahnhof')]

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('Abfahrtsbahnhof.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [90]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_Abfahrtsbahnhof = zugfahrten_df[zugfahrten_df['Abfahrtsbahnhof'].isnull()]

print("Rows where 'Abfahrtsbahnhof' has no values before conversion:")
display(rows_with_missing_Abfahrtsbahnhof)

Rows where 'Abfahrtsbahnhof' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25479,7516839053,3.0,1005,2023-01-05,4783.80,7516839053,2024-06-23 00:25:05,2024-06-23 01:33:05,830.0,312.0,NaN,Hessmouth
25876,7683254886,7.0,1002,2022-02-15,1315.68,7683254886,2024-06-12 21:32:28,2024-06-12 22:45:28,565.0,300.0,NaN,West Kathleenburgh
25980,4122502562,3.0,1004,2023-03-28,4659.26,4122502562,2024-06-21 01:28:38,2024-06-21 05:45:38,851.0,368.0,NaN,Lake Devin
26212,4191950654,9.0,1000,2023-06-17,3872.10,4191950654,2024-06-13 13:15:21,2024-06-13 17:40:21,129.0,266.0,NaN,Mataton


In [91]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer         0
Fahrzeugnummer      5
Buchungskreis       5
Abfahrtsdatum       0
Messung             0
Zug id              0
Abfahrtszeit        0
Ankunftszeit        0
Entfernung km       7
Passagieranzahl     4
Abfahrtsbahnhof     4
Ankunftsbahnhof    45
dtype: int64


In [92]:
zugfahrten_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25099 non-null  float64       
 9   Passagieranzahl  25102 non-null  float64       
 10  Abfahrtsbahnhof  25102 non-null  object        
 11  Ankunftsbahnhof  25061 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(6)
memory usage: 2.5+ MB


### Ankunftsbahnhof  

In [93]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_Ankunftsbahnhof = zugfahrten_df[zugfahrten_df['Ankunftsbahnhof'].isnull()]

print("Rows where 'Ankunftsbahnhof' has no values before conversion:")
display(rows_with_missing_Ankunftsbahnhof)

Rows where 'Ankunftsbahnhof' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
131,4079331200,12.0,XXXX,2021-08-25,2242.64,4079331200,2024-06-08 02:28:47,2024-06-08 05:04:47,662.0,173.0,Tiffanybury,NaN
899,8383166655,11.0,1004,2022-01-25,3769.24,8383166655,2024-06-03 13:52:53,2024-06-03 18:35:53,524.0,118.0,South Christineville,NaN
2440,4486155570,5.0,XXXX,2022-06-09,4598.64,4486155570,2024-06-17 12:35:34,2024-06-17 14:14:34,385.0,489.0,West Rebecca,NaN
2551,2765441613,12.0,1006,2020-06-09,4126.40,2765441613,2024-06-11 19:45:58,2024-06-11 20:54:58,475.0,263.0,Simpsonburgh,NaN
2670,2853823258,6.0,1009,2024-06-07,4882.38,2853823258,2024-06-07 13:27:56,2024-06-07 16:10:56,524.0,315.0,South Annaberg,NaN
2844,8161474014,9.0,1009,2023-11-28,3823.96,8161474014,2024-06-20 05:35:29,2024-06-20 09:52:29,862.0,426.0,North Jonathan,NaN
3709,9937405225,1.0,1003,2023-11-30,4831.36,9937405225,2024-06-09 05:00:05,2024-06-09 07:38:05,145.0,266.0,South Malloryville,NaN
4537,2533957127,9.0,1004,2022-07-08,3508.09,2533957127,2024-06-17 05:58:36,2024-06-17 07:06:36,313.0,116.0,Johnsontown,NaN
4881,3455975229,12.0,1005,2022-08-11,1132.29,3455975229,2024-06-14 05:56:02,2024-06-14 07:11:02,928.0,464.0,Port Donaldbury,NaN
5520,1012674139,6.0,1008,2020-02-15,964.52,1012674139,2024-05-27 18:06:53,2024-05-27 21:23:53,663.0,203.0,Elliottfurt,NaN


In [94]:
# Fill missing values in 'Buchungskreis'
for i in range(len(zugfahrten_df)):
    if pd.isna(zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Ankunftsbahnhof')]):
        if i > 0 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Ankunftsbahnhof')] = zugfahrten_df.iloc[i - 1, zugfahrten_df.columns.get_loc('Ankunftsbahnhof')]
        elif i < len(zugfahrten_df) - 1 and zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Zug id')] == zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Zug id')]:
            zugfahrten_df.iloc[i, zugfahrten_df.columns.get_loc('Ankunftsbahnhof')] = zugfahrten_df.iloc[i + 1, zugfahrten_df.columns.get_loc('Ankunftsbahnhof')]

# Display the DataFrame
print(zugfahrten_df)
zugfahrten_df.to_csv('Ankunftsbahnhof.csv', index=False)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26212  4191950654             9.0          1000    2023-06-17  3872.10   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   
26240  7705851007             3.0          1009    2020-05-20   969.09   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [95]:
# Display the rows where 'Fahrzeugnummer' has no values before conversion
rows_with_missing_Ankunftsbahnhof = zugfahrten_df[zugfahrten_df['Ankunftsbahnhof'].isnull()]

print("Rows where 'Ankunftsbahnhof' has no values before conversion:")
display(rows_with_missing_Ankunftsbahnhof)

Rows where 'Ankunftsbahnhof' has no values before conversion:


,Fahrtnummer,Fahrzeugnummer,Buchungskreis,Abfahrtsdatum,Messung,Zug id,Abfahrtszeit,Ankunftszeit,Entfernung km,Passagieranzahl,Abfahrtsbahnhof,Ankunftsbahnhof
25770,6519204019,15.0,1009,2023-06-26,876.76,6519204019,2024-05-29 14:45:24,2024-05-29 16:40:24,394.0,414.0,East Davidchester,NaN
25919,2655422364,9.0,1002,2022-06-07,3745.63,2655422364,2024-06-06 12:30:44,2024-06-06 17:05:44,986.0,301.0,New Jonathanview,NaN
26163,2362048676,10.0,1008,2020-02-17,3630.75,2362048676,2024-06-05 01:14:59,2024-06-05 04:49:59,67.0,66.0,North Christopher,NaN


In [96]:
# Check the data types
zugfahrten_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25106 entries, 0 to 26240
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   Fahrtnummer      25106 non-null  object        
 1   Fahrzeugnummer   25101 non-null  float64       
 2   Buchungskreis    25101 non-null  object        
 3   Abfahrtsdatum    25106 non-null  datetime64[ns]
 4   Messung          25106 non-null  float64       
 5   Zug id           25106 non-null  object        
 6   Abfahrtszeit     25106 non-null  datetime64[ns]
 7   Ankunftszeit     25106 non-null  object        
 8   Entfernung km    25099 non-null  float64       
 9   Passagieranzahl  25102 non-null  float64       
 10  Abfahrtsbahnhof  25102 non-null  object        
 11  Ankunftsbahnhof  25103 non-null  object        
dtypes: datetime64[ns](2), float64(4), object(6)
memory usage: 2.5+ MB


In [97]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index(['Fahrzeugnummer', 'Buchungskreis', 'Entfernung km', 'Passagieranzahl',
       'Abfahrtsbahnhof', 'Ankunftsbahnhof'],
      dtype='object')
Number of missing values:
 Fahrtnummer        0
Fahrzeugnummer     5
Buchungskreis      5
Abfahrtsdatum      0
Messung            0
Zug id             0
Abfahrtszeit       0
Ankunftszeit       0
Entfernung km      7
Passagieranzahl    4
Abfahrtsbahnhof    4
Ankunftsbahnhof    3
dtype: int64


# Clean again

In [98]:
zugfahrten_df = zugfahrten_df.dropna()

# Display the cleaned DataFrame
print(zugfahrten_df)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
26190  9203259262            12.0          1002    2020-03-09  2005.77   
26192  7195511795            10.0          1009    2022-01-31  3094.97   
26211  0799554257            11.0          1008    2023-05-19  4205.58   
26215  0117443352             3.0          1004    2023-08-01  4294.16   
26235  7772769811             1.0          1002    2022-10-22  3249.77   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [99]:
# Identify columns with null values
columns_with_null = zugfahrten_df.columns[zugfahrten_df.isna().any()]

# Print the columns with null values
print("Columns with null values:")
print(columns_with_null)
print("Number of missing values:\n", zugfahrten_df.isnull().sum())

Columns with null values:
Index([], dtype='object')
Number of missing values:
 Fahrtnummer        0
Fahrzeugnummer     0
Buchungskreis      0
Abfahrtsdatum      0
Messung            0
Zug id             0
Abfahrtszeit       0
Ankunftszeit       0
Entfernung km      0
Passagieranzahl    0
Abfahrtsbahnhof    0
Ankunftsbahnhof    0
dtype: int64


In [100]:
zugfahrten_df = zugfahrten_df.drop_duplicates()

# Display the cleaned DataFrame
print(zugfahrten_df)

      Fahrtnummer  Fahrzeugnummer Buchungskreis Abfahrtsdatum  Messung  \
0      3094127253             1.0          1000    2020-12-29  2321.14   
1      3094127253             2.0          1000    2020-12-29  2361.35   
2      3094127253             3.0          1000    2020-12-29  3078.98   
3      3094127253             4.0          1000    2020-12-29  4063.26   
4      3094127253             5.0          1000    2020-12-29   624.47   
...           ...             ...           ...           ...      ...   
25829  0968001324             5.0          1001    2024-02-02  2078.63   
25950  0291777048            13.0          1008    2023-08-23 -4451.26   
26042  5346820897            17.0          1001    2023-09-14   641.70   
26101  8674828977             2.0          1001    2022-06-26  -781.35   
26173  6972164048             3.0          1007    2020-08-11 -4681.77   

           Zug id        Abfahrtszeit         Ankunftszeit  Entfernung km  \
0      3094127253 2024-06-11 23:31

In [101]:
zugfahrten_df.to_csv('cleaned_data.csv', index=False)